Tilføj forklarende tekst a la 02_heterologous patway (...)

Undersøg enhederne.

Overvej (ikke nødvendigt): 
 - Prøv at ændre mediet på andre måder 
        - E.g. udskift glucose
        - tilføj precursor i mediet.



In [7]:
# Load packages needed
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite

In [8]:
model = read_sbml_model('models/iMM904_progesterone_fixed.xml') # Load model

In [9]:
model.add_boundary(model.metabolites.get_by_id("progesterone_c"), type="demand") # Because it is a reversible reaction, we need to set a demand for production of progesterone 

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x130ba2d70
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


# Maximum theoretical productivity and yield of progesterone on glucose 

In order to asses the model, it can be beneficial to look at the maximum theoretical growth rate, as well as the productivity and yield obtained from the compound of interest. For this, we must first find the nutrients of the growth medium and the quantity of each nutrient.

In [10]:
# See medium compositions
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

It can bee seen that the two limiting exchanges are glucose and oxygen, and that glucose is the sole carbon source of the default growth medium. 

In [13]:
with model: 
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone
    print('Maximum theoretical productivity:', round(progesterone_production,5), 'mmol/gDW*h') # Print max productivity
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # Define D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('Maximum theoretical yield:', round(max_yield,5), 'mmol progesterone/mmol glucose') # Print max yield on glucose

Maximum theoretical productivity: 0.14286 mmol/gDW*h
Maximum theoretical yield: 0.01429 mmol progesterone/mmol glucose


## Maximum theoretical productivity and yield in Cmole

To account for cabon molecules, instead of mass, we can convert the productivity and yield to Cmole.

In [11]:
# Define conversion factors
Cmole_glucose = 6
Cmole_progesterone = 21

In [14]:
progesterone_production_Cmole = (progesterone_production)/(Cmole_progesterone) # Max productivity in Cmole
max_yield_Cmole = (max_yield*Cmole_progesterone)/(Cmole_glucose) # Max yield in Cmole
print('Maximum theoretical productivity:', round(progesterone_production_Cmole,5), 'Cmole/gDW*h') # Print max productivity in Cmole
print('Maximum theoretical yield:', round(max_yield_Cmole,5), 'Cmole progesterone/Cmole glucose') # Print max yield on glucose in Cmole

Maximum theoretical productivity: 0.0068 Cmole/gDW*h
Maximum theoretical yield: 0.05 Cmole progesterone/Cmole glucose


# Increasing glucose availability in the growth medium

Ass seen before, one of the rate limiting compounds is glucose. As glucose is only available in a low quantity in the medium, it could be interesting to increase the concentration of glucose in the medium and see how it affects the theoretical growth rate of the strain.

In [22]:
# Define D-glucose exchange reaction
D_glucose_exchange = model.reactions.get_by_id('EX_glc__D_e')

In [25]:
# Simulate the model at different glucose concentrations with biomass as objective
medium = model.medium
with model:
    objective_value = model.reactions.BIOMASS_SC5_notrace # Objective is to maximise growth
    growth = model.optimize().objective_value # Simulate model 
    print('At glucose concentration:', format(medium[D_glucose_exchange.id]))
    print('Maximum theoretical growth rate:', round(growth,5), '/h') # Since biomass describes metabolite composition, the unit is now only /h 
    print('')

    medium[D_glucose_exchange.id] = 500 # Set D-glucose to half-max (500)
    model.medium = medium
    growth = model.optimize().objective_value 
    print('At glucose concentration:', format(medium[D_glucose_exchange.id]))
    print('Maximum theoretical growth rate:', round(growth,5), '/h')
    print('')

    medium[D_glucose_exchange.id] = 1000 # Set D-glucose to max (1000)
    model.medium = medium
    growth = model.optimize().objective_value 
    print('At glucose concentration:', format(medium[D_glucose_exchange.id]))
    print('Maximum theoretical growth rate:', round(growth,5), '/h')



At glucose concentration: 10.0
Maximum theoretical growth rate: 0.28787 /h

At glucose concentration: 500
Maximum theoretical growth rate: 11.06243 /h

At glucose concentration: 1000
Maximum theoretical growth rate: 21.89736 /h


It can be seen that the growth rate is increased when the availability of glucose in the growth medium is increased. Hence, it could be interesting to simulate the effect of increasing glucose in the medium on the production of progesterone.

## The effect on progesterone production by increasing glucose availability

Now, we will simulate the effect that increasing the glucose concentration in the medium has on the production of progesterone.

In [96]:
# Simulate the model with increasing glucose concentrations with progesterone production as objective
medium = model.medium
with model: 
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # Define D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('At glucose concentration:', format(medium[D_glucose_exchange.id]))
    print('Maximum theoretical productivity:', round(progesterone_production,5), 'mmol/gDW*h')
    print('Maximum theoretical yield:', round(max_yield,5), 'mmol progesterone/mmol glucose')
    print('')

    medium[D_glucose_exchange.id] = 500 # Set D-glucose to half-max
    model.medium = medium 

    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone and new glucose flux
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('At glucose concentration:', format(medium[D_glucose_exchange.id]))
    print('Maximum theoretical productivity:', round(progesterone_production,5), 'mmol/gDW*h')
    print('Maximum theoretical yield:', round(max_yield,5), 'mmol progesterone/mmol glucose')
    print('')

    medium[D_glucose_exchange.id] = 1000 # Set D-glucose to max
    model.medium = medium 

    progesterone_production = model.optimize().objective_value # Define new model with maximum production of progesterone and new glucose flux
    D_glucose_flux = model.reactions.EX_glc__D_e.flux # D-Glucose flux
    max_yield = progesterone_production / (-1 * D_glucose_flux) # Calculate max yield on glucose
    print('At glucose concentration:', format(medium[D_glucose_exchange.id]))
    print('Maximum theoretical productivity:', round(progesterone_production,5), 'mmol/gDW*h')
    print('Maximum theoretical yield:', round(max_yield,5), 'mmol progesterone/mmol glucose')

At glucose concentration: 10.0
Maximum theoretical productivity: 0.14286 mmol/gDW*h
Maximum theoretical yield: 0.01429 mmol progesterone/mmol glucose

At glucose concentration: 500
Maximum theoretical productivity: 0.14286 mmol/gDW*h
Maximum theoretical yield: 0.00029 mmol progesterone/mmol glucose

At glucose concentration: 1000
Maximum theoretical productivity: 0.14286 mmol/gDW*h
Maximum theoretical yield: 0.00014 mmol progesterone/mmol glucose


Interestingly, it appears as if the productivity of progesterone does not change when the amount of glucose is increased. Instead, the productivity is constant, thus, the yield is lowered as the glucose concentration increases and the productivity remains steady.  Therefore, only increasing the concentration of glucose is not a feasible idea for increasing progesterone yield - something else must be changed to increase the productivity of progesterone, first then, increasing the glucose concentration might be beneficial.